<a href="https://colab.research.google.com/github/venkat0778/sentiment_analysis/blob/main/sentimental_analysis_using_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install azure-storage-blob


In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

plt.style.use('ggplot')

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from bs4 import BeautifulSoup

In [ ]:

connection_string = 'DefaultEndpointsProtocol=https;AccountName=venkat778;AccountKey=PJ1Zu8Ml5u6uZ9vzusOWecpg4dGIS1fVUdHICMuZP4jgprkE7rdLwUrF8+QgbCHksA3hGnI2yVY2+AStoQm8Qw==;EndpointSuffix=core.windows.net'
container_name = 'amazonreviews'
blob_name = 'Amazon_Reviews.csv'  # The name of your blob (CSV file)

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a BlobClient to interact with the specific blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob data into a bytes object
downloaded_blob = blob_client.download_blob().readall()

# Convert the bytes object to a pandas DataFrame
df = pd.read_csv(io.BytesIO(downloaded_blob))

In [ ]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,11555559,R1QXC7AHHJBQ3O,B00IKPX4GY,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Five Stars,Great love it,31-08-2015,1
1,US,31469372,R175VSRV6ZETOP,B00IKPYKWG,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,3,0,0,N,N,Lots of ads Slow processing speed Occasionally...,Lots of ads<br />Slow processing speed<br />Oc...,31-08-2015,0
2,US,26843895,R2HRFF78MWGY19,B00IKPW0UA,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Well thought out device,Excellent unit. The versatility of this table...,31-08-2015,1
3,US,19844868,R8Q39WPKYVSTX,B00LCHSHMS,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,4,0,0,N,N,Not all apps/games we were looking forward to ...,I bought this on Amazon Prime so I ended up bu...,31-08-2015,1
4,US,1189852,R3RL4C8YP2ZCJL,B00IKPZ5V6,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Five Stars,All Amazon products continue to meet my expect...,31-08-2015,1


In [ ]:
print(df.shape)
df = df.head(500)
print(df.shape)
df.head(3)

(30846, 16)
(500, 16)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,11555559,R1QXC7AHHJBQ3O,B00IKPX4GY,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Five Stars,Great love it,31-08-2015,1
1,US,31469372,R175VSRV6ZETOP,B00IKPYKWG,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,3,0,0,N,N,Lots of ads Slow processing speed Occasionally...,Lots of ads<br />Slow processing speed<br />Oc...,31-08-2015,0
2,US,26843895,R2HRFF78MWGY19,B00IKPW0UA,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Well thought out device,Excellent unit. The versatility of this table...,31-08-2015,1


In [ ]:
# function to remove html elements from the reviews
def removeHTML(raw_text):
    clean_HTML = BeautifulSoup(raw_text, 'lxml').get_text()
    return clean_HTML

In [ ]:
# function to remove special characters and numbers from the reviews4961
def removeSpecialChar(raw_text):
    clean_SpecialChar = re.sub("[^a-zA-Z]", " ", raw_text)
    return clean_SpecialChar

In [ ]:
# function to convert all reviews into lower case
def toLowerCase(raw_text):
    clean_LowerCase = raw_text.lower().split()
    return( " ".join(clean_LowerCase))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# function to remove stop words from the reviews
def removeStopWords(raw_text):
    stops = set(stopwords.words("english"))
    words = [w for w in raw_text if not w in stops]
    return( " ".join(words))

In [ ]:
# X_training clean set
example = []

for val in df['review_body']:
    val = removeHTML(val)
    val = removeSpecialChar(val)
    val = toLowerCase(val)
    removeStopWords(val)
    example.append(val)

<ipython-input-10-c2a805237a43>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_HTML = BeautifulSoup(raw_text, 'lxml').get_text()


In [ ]:
print(example)

['great love it', 'lots of adsslow processing speedoccasionally shuts down appswifi keeps having authentication issueswas cheap for a tablet and now i know why', 'excellent unit the versatility of this tablet besides being competitively priced is a solution to the elderly poor eyesight and physical disabilities associated with age and using the supporting add on features allows the user to stay in touch with our changing world a realistic add on keyboard that you can see and use i cannot wait to use my fire hd to show computer created work sheets and class instructions to my students hmdi good by to copiers and reams of paper farwell to those costly printer inks oh yes i have much more to gain back just takes a little effort to learn more open a book and read noel', 'i bought this on amazon prime so i ended up buying the gb one for the camera is okay and i love the edit features you can do to the pictures amazon netflix huliu all run smoothly on this tablet the internet runs a lot fast

In [ ]:
tokens = nltk.word_tokenize(example)
tagged = nltk.pos_tag(tokens)
entities = nltk.chunk.ne_chunk(tagged)

TypeError: expected string or bytes-like object

In [ ]:
example = df['review_body'][49]
print(example)

tokens = nltk.word_tokenize(example)
tagged = nltk.pos_tag(tokens)
entities = nltk.chunk.ne_chunk(tagged)

This is the first real tablet I've used with any success. I was once given a ProntoTec 7&#34; tablet and it was so useless, I was put off on tablets altogether. But the Kindle Fire actually works, and works well. It's quick, responds accurately to my touch, and makes an excellent Kindle ebook reader. I've been reading like crazy since I got it. It also performs fine for simple things like checking email and getting notices from my gmail calendar. For other uses, like games, or anything that involves text input, I'd rather be at my desktop with a big screen and good keyboard. That's not a knock on the Fire, it's just my preference. The point is that I haven't been downloading apps and using them. Other than sitting in my easy chair and reading, I can't think of anything I'd rather do on a tablet than on a desktop PC.<br /><br />I once read that Amazon sells the Kindles at below cost in the hope the buyers will spend money on the books, movies, videos, subscription services, etc. I don't

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Run for Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

RuntimeError: The expanded size of the tensor (682) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 682].  Tensor sizes: [1, 514]

In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['review_body']
        myid = row['customer_id']
        #myid = row['Id']
        roberta_result = polarity_scores_roberta(text)
        #both = { **roberta_result}
        res[myid] = roberta_result
    except RuntimeError:
        print(f'Broke for id ')

  0%|          | 0/500 [00:00<?, ?it/s]

Broke for id 
Broke for id 


In [ ]:
roberta_result

{'roberta_neg': 0.035755094,
 'roberta_neu': 0.60179996,
 'roberta_pos': 0.36244494}

In [ ]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'customer_id'})
results_df = results_df.merge(df, how='left')

In [ ]:
results_df

,customer_id,roberta_neg,roberta_neu,roberta_pos,marketplace,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,11555559,0.006225,0.029002,0.964772,US,R1QXC7AHHJBQ3O,B00IKPX4GY,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Five Stars,Great love it,31-08-2015,1
1,31469372,0.861340,0.129977,0.008683,US,R175VSRV6ZETOP,B00IKPYKWG,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,3,0,0,N,N,Lots of ads Slow processing speed Occasionally...,Lots of ads<br />Slow processing speed<br />Oc...,31-08-2015,0
2,26843895,0.018098,0.096249,0.885653,US,R2HRFF78MWGY19,B00IKPW0UA,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Well thought out device,Excellent unit. The versatility of this table...,31-08-2015,1
3,19844868,0.131736,0.271661,0.596603,US,R8Q39WPKYVSTX,B00LCHSHMS,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,4,0,0,N,N,Not all apps/games we were looking forward to ...,I bought this on Amazon Prime so I ended up bu...,31-08-2015,1
4,1189852,0.003927,0.041826,0.954247,US,R3RL4C8YP2ZCJL,B00IKPZ5V6,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Five Stars,All Amazon products continue to meet my expect...,31-08-2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,50186353,0.002791,0.018927,0.978282,US,R1DQ7ESAH5Z8JI,B00LCHOCZY,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,N,I love my kindle,"I love this kindle, I check my emails watch mo...",20-08-2015,1
494,13640575,0.002181,0.005925,0.991894,US,R3F6GBOL08A0HU,B00LCHS1FG,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,Y,Love is my Kindle Fire.,It's the best!! I bring it with me on my comm...,20-08-2015,1
495,14896096,0.026720,0.132836,0.840444,US,R1KBU0LIQ9T0IE,B00IKPZ5V6,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,5,0,0,N,N,Five Stars,Great buy! Still have a lot to learn,20-08-2015,1
496,5439056,0.910953,0.078512,0.010535,US,R3LVC9R6D3EKQ1,B00IKPYKWG,2693241,"Fire HD 7, 7"" HD Display, Wi-Fi, 8 GB",PC,2,0,0,N,N,One of those products that reminds you of the ...,One of those products that reminds you of the ...,20-08-2015,0
